In [1]:
## preparation
## all imports
import requests
import re 
from bs4 import BeautifulSoup
import time

import json
import numpy as np
import urllib.request

import pandas as pd

In [241]:
def get_data(product_link):
    response = requests.get(product_link)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    
    product_id = re.findall(R'P[0-9]{3,6}',product_link)[0]

    # Get brand name and product name
    for brand_and_name in soup.find_all('h1', class_='css-a1jw00'):
        names = [names.get_text() for names in brand_and_name.find_all('span')]
        brand = names[0]
        prd_name = names[1]

    # Get Category and price (use json to get categories and price),
    # Because it would give us sub categories and offer price.
    # But if that doesn't work, just get main category and price box.
    dic_json = soup.find_all(attrs={"type": "application/ld+json"})
    try:
        json_category = json.loads(dic_json[0].get_text())
        category_lst = [item['item']['name']
                    for item in json_category['itemListElement']]
        category = '/'.join(category_lst)
        price = json.loads(dic_json[1].get_text())['offers'][0]['price']

    except:
        category = [cate.get_text()
                    for cate in soup.find_all('ol', class_='css-1doqpel')]
        price = soup.find('div', attrs={"data-comp": "Price Box"}).get_text()

    # Get item number and size
    size_and_item = soup.find(attrs={"data-comp": "SizeAndItemNumber Box"})    
    ## If there's only item number and no size info, it will be out of index
    try:
        item_num = size_and_item.contents[1].split(' ')[1] 
    except:
        item_num = size_and_item.contents[0].split(' ')[1]
    ## If there's no size at SizeAndItemNumber Box, then try use description area to see.
    try:
        size = size_and_item.span.contents[0].split('SIZE ')[1]
    except:
        try:
            size = soup.find("span", attrs={"class": "css-12wl10d"}).contents[-1]
        except:
            size = 'na'
            
    # Get love counts
    try:
        love_counts = soup.find('span', attrs={"data-at": "product_love_count"}).get_text()
    except:
        love_counts = 'na'
    
    # review nums and ratings
    link_json = soup.find(attrs={"id": "linkJSON"})
    json_str = str(link_json)
    ratings = re.findall(R'\"rating\"\:(.*?)\,',json_str)
    reviews = re.findall(R'\"reviews\"\:(.*?)\,',json_str)
    try:
        rating = ratings[0]
    except:
        rating = 'na'
    try:
        reviews_count = reviews[0]
    except:
        reviews_count = 'na'
    
    dic1 = {}
    dic1['category'] = category
    dic1['product_name']= prd_name
    dic1['brand'] = brand
    dic1['price'] = price
    dic1['Product_Id'] = product_id
    dic1['size'] = size
    dic1['item_num'] = item_num
    dic1['love_count'] = love_counts
    dic1['rating'] = rating
    dic1['reviews_count'] = reviews_count
    dic1['link'] = product_link
    return dic1

In [242]:
get_data(product_links[1])

{'category': 'Fragrance/Men/Cologne',
 'product_name': 'Colonia',
 'brand': 'Acqua Di Parma',
 'price': '63.0',
 'Product_Id': 'P163604',
 'size': '0.7 oz/ 20 mL',
 'item_num': '2044816',
 'love_count': '2528',
 'rating': '4.3067',
 'reviews_count': '75',
 'link': 'https://www.sephora.com/product/colonia-P163604?icid2=products'}

In [244]:
#Test: get first 5 product info
products = open('product_link.txt')
i = 0
frame = []
for item in products:
    link = item.rstrip()
    product_data = get_data(link)
    df = pd.DataFrame(product_data,index=[i])
    frame.append(df)
    i+=1
    time.sleep(np.random.random()*3)
    if i == 5:
        break
result = pd.concat(frame)
result
#result.to_excel('products.xlsx')

,category,product_name,brand,price,Product_Id,size,item_num,love_count,rating,reviews_count,link
0,Fragrance/Value & Gift Sets/Perfume Gift Sets,Blu Mediterraneo MINIATURE Set,Acqua Di Parma,63.0,P443401,5 x 0.16oz/5mL,2218774,2588,4,4,https://www.sephora.com/product/blu-mediterran...
1,Fragrance/Men/Cologne,Colonia,Acqua Di Parma,63.0,P163604,0.7 oz/ 20 mL,2044816,2528,4.3067,75,https://www.sephora.com/product/colonia-P16360...
2,Fragrance/Women/Perfume,Mirto di Panarea,Acqua Di Parma,64.0,P307804,2.5 oz/ 74 mL,1417617,2622,4.6364,22,https://www.sephora.com/product/blu-mediterran...
3,Fragrance/Women/Perfume,Rosa Nobile,Acqua Di Parma,68.0,P388670,3.4 oz/ 101 mL,1638832,4569,4.5652,69,https://www.sephora.com/product/rosa-nobile-P3...
4,Fragrance/Value & Gift Sets/Perfume Gift Sets,Colonia Miniature Set,Acqua Di Parma,69.0,P443400,5 x 0.16oz/5mL,2218766,779,3.5,2,https://www.sephora.com/product/colonia-miniat...


In [245]:
## How many products? 
products = open('product_link.txt')
count = 0
for item in products:
    count += 1
print('products #', count)

products # 3091


### There are over 3000 products on Sephora website. 
### But It seems that Sephora has set limits for request times each IP address and each day. 
### Therefore, we can try to scape all the product information using different IP addresses or different days.

In [5]:
## 1st time
products = open('product_link.txt')
product_links = []
for item in products:
    link = item.rstrip()
    product_links.append(link)
    

frame1 = []
i = 0
for link in product_links[:5]: ## replace 5 with 5000
    product_data = get_data(link)
    df = DataFrame(product_data,index=[i])
    frame1.append(df)
    i+=1
    time.sleep(np.random.random()*3)

result1 = pd.concat(frame1)
result1


,Product_Id,brand,category,item_num,link,love_count,price,product_name,rating,reviews_count,size
0,P388670,Acqua Di Parma,Fragrance/Women/Perfume/,2044774,https://www.sephora.com/product/rosa-nobile-P3...,3932,68.0,Rosa Nobile,4.4643,28,"0.7 oz, 20 mL"
1,P443401,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets/,2218774,https://www.sephora.com/product/blu-mediterran...,1347,63.0,Blu Mediterraneo MINIATURE Set,5,2,5 x 0.16oz/5mL
2,P163604,Acqua Di Parma,Fragrance/Men/Cologne/,967083,https://www.sephora.com/product/colonia-P16360...,2272,63.0,Colonia,4.2778,72,1.7 oz/ 50 mL
3,P375388,Acqua Di Parma,Fragrance/Women/Perfume/,1417567,https://www.sephora.com/product/blu-mediterran...,2257,64.0,Arancia di Capri,4.24,25,5 oz/ 148 mL
4,P431439,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets/,2044766,https://www.sephora.com/product/le-nobili-coll...,975,68.0,Le Nobili Collection Set,4.1,10,3 x 0.51 oz/ 15 mL


In [6]:
## 2nd time
frame2 = []
i = 5 ## replace 5 with 5000
for link in product_links[5:10]: ## replace [5:10] with [5000:]
    product_data = get_data(link)
    df = DataFrame(product_data,index=[i])
    frame2.append(df)
    i+=1
    time.sleep(np.random.random()*3)
result2 = pd.concat(frame2)
result2

,Product_Id,brand,category,item_num,link,love_count,price,product_name,rating,reviews_count,size
5,P413669,Acqua Di Parma,Fragrance/Women/Perfume/,1886621,https://www.sephora.com/product/peonia-nobile-...,1793,68.0,Peonia Nobile,4.2353,17,1.7 oz/ 50 mL
6,P443400,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets/,2218766,https://www.sephora.com/product/colonia-miniat...,394,69.0,Colonia Miniature Set,na,na,5 x 0.16oz/5mL
7,P307801,Acqua Di Parma,Fragrance/Women/Perfume/,1417591,https://www.sephora.com/product/blu-mediterran...,1990,64.0,Fico di Amalfi,4.4194,31,2.5 oz/ 74 mL
8,P307804,Acqua Di Parma,Fragrance/Women/Perfume/,1417617,https://www.sephora.com/product/blu-mediterran...,2041,64.0,Mirto di Panarea,4.6,20,2.5 oz/ 74 mL
9,P307802,Acqua Di Parma,Fragrance/Women/Perfume/,2044758,https://www.sephora.com/product/blu-mediterran...,1681,64.0,Bergamotto di Calabria,4.5,16,1 oz/ 30 mL


In [7]:
result = pd.concat([result1,result2])
result

,Product_Id,brand,category,item_num,link,love_count,price,product_name,rating,reviews_count,size
0,P388670,Acqua Di Parma,Fragrance/Women/Perfume/,2044774,https://www.sephora.com/product/rosa-nobile-P3...,3932,68.0,Rosa Nobile,4.4643,28,"0.7 oz, 20 mL"
1,P443401,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets/,2218774,https://www.sephora.com/product/blu-mediterran...,1347,63.0,Blu Mediterraneo MINIATURE Set,5,2,5 x 0.16oz/5mL
2,P163604,Acqua Di Parma,Fragrance/Men/Cologne/,967083,https://www.sephora.com/product/colonia-P16360...,2272,63.0,Colonia,4.2778,72,1.7 oz/ 50 mL
3,P375388,Acqua Di Parma,Fragrance/Women/Perfume/,1417567,https://www.sephora.com/product/blu-mediterran...,2257,64.0,Arancia di Capri,4.24,25,5 oz/ 148 mL
4,P431439,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets/,2044766,https://www.sephora.com/product/le-nobili-coll...,975,68.0,Le Nobili Collection Set,4.1,10,3 x 0.51 oz/ 15 mL
5,P413669,Acqua Di Parma,Fragrance/Women/Perfume/,1886621,https://www.sephora.com/product/peonia-nobile-...,1793,68.0,Peonia Nobile,4.2353,17,1.7 oz/ 50 mL
6,P443400,Acqua Di Parma,Fragrance/Value & Gift Sets/Perfume Gift Sets/,2218766,https://www.sephora.com/product/colonia-miniat...,394,69.0,Colonia Miniature Set,na,na,5 x 0.16oz/5mL
7,P307801,Acqua Di Parma,Fragrance/Women/Perfume/,1417591,https://www.sephora.com/product/blu-mediterran...,1990,64.0,Fico di Amalfi,4.4194,31,2.5 oz/ 74 mL
8,P307804,Acqua Di Parma,Fragrance/Women/Perfume/,1417617,https://www.sephora.com/product/blu-mediterran...,2041,64.0,Mirto di Panarea,4.6,20,2.5 oz/ 74 mL
9,P307802,Acqua Di Parma,Fragrance/Women/Perfume/,2044758,https://www.sephora.com/product/blu-mediterran...,1681,64.0,Bergamotto di Calabria,4.5,16,1 oz/ 30 mL


In [ ]:
## Save the result as xlsx. 
result.to_excel('products.xlsx')

In [2]:
# Old version get_data function

def get_data(link):
    wb_data = requests.get(link)
    soup = bs4.BeautifulSoup(wb_data.text,'lxml')
    
    ## para1 is to get info of sort,price,product name and brand
    para1 = soup.find_all(attrs={"type": "application/ld+json"})
    sort = ''
    for content1 in para1[:2]:
        content1 = content1.string
        try:
            content1_dict = json.loads(content1)
        except:
            continue
        try:
            content1_lst = content1_dict['itemListElement']
            for item in content1_lst:
                sort= sort+item['item']['name']+ '/'
        except:
            price = content1_dict['offers'][0]['price'] 
            product_name = content1_dict['name']          
            brand = content1_dict['brand']
    
    ## para2 is to get info of size and item number
    para2 = soup.find(attrs={"class": "css-1qf1va5"})
    para2 = str(para2)
    try:
        size = re.findall(R'SIZE.+?\>(.*?)\<span',para2)[0]
    except:
        size ='na'
    try:
        item_num = re.findall(R'ITEM.+?\>(.*?)\</div\>',para2)[0]
    except:
        item_num = 'na'
    
    ## para3 is to get info of love counts
    para3 = soup.find(attrs={"data-at": "product_love_count"})
    para3 = str(para3)
    try:
        love_count = re.findall(R'\>([0-9].+)\</span\>',para3)[0]
    except:
        love_count = 'na'
    
    ## para4 is to get info of rating and review counts
    para4 = soup.find(attrs={"id": "linkJSON"})
    para4 = str(para4)
    ratings = re.findall(R'\"rating\"\:(.*?)\,',para4)
    reviews = re.findall(R'\"reviews\"\:(.*?)\,',para4)
    
    
    try:
        rating = ratings[0]
    except:
        rating = 'na'
    
    try:
        reviews_count = reviews[0]
    except:
        reviews_count = 'na'
    
    if size == []:
            size = 'na'
    if item_num ==[]:
        item_num = 'na'
    if love_count == []:
        love_count = 'na'
     
    dic1 = {}
    dic1['category'] = sort
    dic1['product_name']= product_name
    dic1['brand'] = brand
    dic1['price'] = price
    dic1['Product_Id'] = re.findall(R'P[0-9]{3,6}',link)
    dic1['size'] = size
    dic1['item_num'] = item_num
    dic1['love_count'] = love_count
    dic1['rating'] = rating
    dic1['reviews_count'] = reviews_count
    dic1['link'] = link
    
    return dic1